In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import time
# Plotting libraries
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from aind_vr_foraging_analysis.utils.plotting import plotting_friction_experiment as f
from aind_vr_foraging_analysis.utils.parsing import data_access, parse, AddExtraColumns
import aind_vr_foraging_analysis.utils.plotting as plotting


sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
color4='yellow'
odor_list_color = [color1, color2, color3, color4]

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = r'Z:/scratch/vr-foraging/data/'
results_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\manuscript\results\figures'
data_path = r'C:\git\Aind.Behavior.VrForaging.Analysis\data'

palette = {
    'control': 'darkgrey',  # Red
    'friction_high': "#1e4110",  # Purple
    'friction_med': "#120c8a",  # Lighter Purple
    'friction_low': '#9e9ac8',  # Lightest Purple
    'distance_extra_short': 'crimson',  # Blue
    'distance_short': 'pink',  # Lighter Blue
    'distance_extra_long': '#fd8d3c',  # Yellow
    'distance_long': '#fdae6b'  # Lighter Yellow
}

color_dict_label = {'Ethyl Butyrate': color1, 'Alpha-pinene': color2, 'Amyl Acetate': color3, 
                    '2-Heptanone' : color2, 'Methyl Acetate': color1, 'Fenchone': color3, '2,3-Butanedione': color4,
                    'Methyl Butyrate': color1, '60': color2, '90': color1, "0": color3}

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import patches as mpatches
import json

In [ ]:
mouse_list = ['788641','789911', '789919', '789913', '789918', '789908',
              '754570','754579','754567','754580','754559','754560','754577',
              '754566','754571','754574','754575', 
              '754582','745302','745301','745305'
              ]

results_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\Conferences\Lakes 2025\figures'


In [ ]:
date_string = "2023-5-8"
df_cum_velocity = pd.DataFrame()
for mouse in mouse_list:
    df_velocity = pd.DataFrame()
    session_paths = data_access.find_sessions_relative_to_date(
        mouse=mouse,
        date_string=date_string,
        when='on_or_after'
    )

    for session_path in session_paths:
        print(mouse, session_path)
        
        try:
            all_epochs, stream_data, data = data_access.load_session(
                session_path
            )
        except:
            print(f"Error loading {session_path.name}")
            continue
        
        
        # Remove segments where the mouse was disengaged
        last_engaged_patch = all_epochs['patch_number'][all_epochs['skipped_count'] >= 5].min()
        if pd.isna(last_engaged_patch):
            last_engaged_patch = all_epochs['patch_number'].max() - 5
        all_epochs['engaged'] = (all_epochs['patch_number'] <= last_engaged_patch) | (all_epochs.patch_number < 20)
        all_epochs['mouse'] = mouse
        all_epochs['session'] = session_path.name
        all_epochs = all_epochs.loc[all_epochs['engaged']==1]
        all_epochs["relative_visit"] = all_epochs.groupby(["patch_number"])["site_number"].transform(lambda x: x - x.max())
        
        reward_sites = all_epochs.loc[(all_epochs.label == 'OdorSite')]
        
        for aligned in ['index', 'choice_cue_time']:
            trial_collection = plotting.trial_collection(
                reward_sites,  stream_data.encoder_data, aligned=aligned, window=[-2, 6],
            )

            trial_collection['aligned'] = aligned
            test = trial_collection.loc[(trial_collection.is_reward == False)&(trial_collection.is_choice == 1)].groupby(['mouse', 'session', 'relative_visit', 'patch_label','times', 'aligned']).speed.mean().reset_index()
            df_velocity = pd.concat([test, df_velocity], axis=0, ignore_index=True)

    df_cum_velocity = pd.concat([df_cum_velocity, df_velocity], axis=0, ignore_index=True)

In [ ]:
sns.relplot(
    data=df_cum_velocity.loc[(df_cum_velocity.aligned == 'index') & (df_cum_velocity.relative_visit.isin([-4,-3,-2, -1]))],
    x='times',
    y='speed',
    hue='relative_visit',
    col='mouse',
    kind='line',
    errorbar=None,
    col_wrap=4  # adjust number of columns
)


In [ ]:
df_cum_velocity.to_csv(os.path.join(data_path, 'velocity_aligned_to_things.csv'))